# 📑 Evaluación de WhyXrayCLIP en ROCO

Este notebook replica el flujo ISA con **WhyXrayCLIP ViT-L/14** sobre radiografías de tórax filtradas del dataset **ROCO**.

Modelo actual: **WhyXrayCLIP (ViT-L/14)**
Dataset: **ROCO (Radiology Objects in COntext)** filtrado a captions `chest x-ray|lung`
Tarea: **ISA (Image-Sentence Alignment)**
---



## Clonar repositorio

In [11]:
# 📌 Código
REPO_URL  = "https://github.com/Alberto-97sc/mmshap_medclip.git"
LOCAL_DIR = "codex/adapt-whyxrayclip-model-to-modular-repo"
BRANCH    = "main"

%cd /content
import os, shutil, subprocess, sys

if not os.path.isdir(f"{LOCAL_DIR}/.git"):
    # No está clonado aún
    !git clone $REPO_URL $LOCAL_DIR
else:
    # Ya existe: actualiza a la última versión del remoto
    %cd $LOCAL_DIR
    !git fetch origin
    !git checkout $BRANCH
    !git reset --hard origin/$BRANCH
%cd $LOCAL_DIR
!git rev-parse --short HEAD

/content
Cloning into 'codex/adapt-whyxrayclip-model-to-modular-repo'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 135 (delta 28), reused 27 (delta 18), pack-reused 67 (from 1)
Receiving objects: 100% (135/135), 1.13 MiB | 11.90 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/codex/adapt-whyxrayclip-model-to-modular-repo
604c76a


## Instalar dependencias y montar google drive

In [1]:
from google.colab import drive; drive.mount('/content/drive')

# === Instalar en modo editable (pyproject.toml) ===
%pip install -e /content/mmshap_medclip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Obtaining file:///content/mmshap_medclip
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mmshap_medclip (pyproject.toml) ... done
  Created wheel for mmshap_medclip: filename=mmshap_medclip-0.1.0-0.editable-py3-none-any.whl size=3995 sha256=0e1e3e0689cc67cf64b9db60a4297c45fd20302cbc8399e0b131060b98593677
  Stored in directory: /tmp/pip-ephem-wheel-cache-wnp3re5t/wheels/b3/1d/7e/d38403be6961da2a3fb9229d1eb25770838e383069b89a1567
Successfully built mmshap_medclip
  Attempting uninstall: mmshap_medclip
    Found existing installation: mmshap_medclip 0.1.0
    Uninstalling mmshap_medclip-0.1.0:
      Successfully uninstalled mmshap_medclip-0.1.0


## Cargar modelos y datos

In [3]:
# 📌 Código
CFG_PATH="/content/mmshap_medclip/configs/roco_isa_whyxrayclip.yaml"

# Asegura que cfg, device, dataset y model estén listos en esta sesión
if not all(k in globals() for k in ("cfg", "device", "dataset", "model")):
    from mmshap_medclip.io_utils import load_config
    from mmshap_medclip.devices import get_device
    from mmshap_medclip.registry import build_dataset, build_model

    cfg = load_config(CFG_PATH)
    device  = get_device()
    dataset = build_dataset(cfg["dataset"])
    model_cfg = cfg.get("model", {}) # Get model config, default to empty dict
    model_params = model_cfg.get("params") if model_cfg.get("params") is not None else {} # Get model params, default to empty dict if None
    model_name = model_cfg.get("name")
    model = build_model({"name": model_name, "params": model_params}, device=device)


print("OK → len(dataset) =", len(dataset), "| device =", device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OK → len(dataset) = 457 | device = cuda


## Ejecutar SHAP en una muestra

In [4]:
# 📌 Código
from mmshap_medclip.tasks.isa import run_isa_one

muestra = 154
sample  = dataset[muestra]
image, caption = sample['image'], sample['text']

res = run_isa_one(model, image, caption, device, explain=True, plot=True)
print(f"logit={res['logit']:.4f}  TScore={res['tscore']:.2%}  IScore={res['iscore']:.2%}")



RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [ ]:
# ==== WhyXrayCLIP + ROCO filtrado (mini pipeline) ====
import random
from IPython.display import display

assert 'dataset' in globals() and 'model' in globals() and 'device' in globals(), 'Ejecuta las celdas previas primero.'
random.seed(42)
idx = random.randrange(len(dataset))
sample = dataset[idx]
image, caption, meta = sample['image'], sample['text'], sample['meta']

print(f'Índice seleccionado: {idx}')
print(f"Nombre en CSV: {meta.get("filename")} | Ruta ZIP: {meta.get("zip_path")}")
print('Caption original:')
print(caption)
display(image)

from mmshap_medclip.tasks.isa import run_isa_one
res = run_isa_one(model, image, caption, device, explain=True, plot=True)
print(f"logit={res['logit']:.4f}  TScore={res['tscore']:.2%}  IScore={res['iscore']:.2%}")
fig = res.get('fig')
if fig is not None:
    display(fig)
